# Softmax


Это задание аналогично SVM. Здесь вы:

- реализуете полностью векторизованную **функцию потерь** для SVM
- создадите векторизованное выражение для **аналитического градиента**
- **проверите свою реализацию** с помощью числового градиента
- используете оценочную выборку для **настройки скорости обучения и силы регуляризации**
- **оптимизируете** функцию потерь с помощью **SGD**
- **визуализируете** окончательные веса

In [ ]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Загружаем датасет CIFAR-10 и предварительно обрабатываем данные.
    Здесь используются те же действия, что и в SVM, но сжатые в одну функцию.  
    """
    # Загружаем исходные данные CIFAR-10
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    # Очищаем переменные, чтобы данные можно было загружать много раз 
    try:
       del X_train, y_train
       del X_test, y_test
       print('Очистка предыдущих данных.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Разделяем данные на подвыборки
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Преобразуем данные в строки
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Нормализация: вычитаем среднее изображение
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # Добавляем смещение и преобразуем в столбцы
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Используем функцию выше, чтобы подготовить все данные
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Объём обучающей выборки: ', X_train.shape)
print('Число обучающих меток: ', y_train.shape)
print('Объём оценочной выборки: ', X_val.shape)
print('Число оценочных меток: ', y_val.shape)
print('Объём тестовой выборки: ', X_test.shape)
print('Число тестовых меток: ', y_test.shape)
print('Объём данных dev: ', X_dev.shape)
print('Число меток dev: ', y_dev.shape)

## Классификатор Softmax
Чтобы выполнить эту секцию, реализуйте ваш код в файле **cs231n/classifiers/softmax.py**. 


In [ ]:
# Сначала реализуйте наивный метод поиска потерь softmax с циклами.
# Откройте файл cs231n/classifiers/softmax.py и напишите функцию
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Генерируем рандомную матрицу весов softmax и считаем потери.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# Грубая проверка: наши потери должны быть близки к -log(0.1).
print('Потери: %f' % loss)
print('Грубая проверка: %f' % (-np.log(0.1)))

In [ ]:
# Завершите функцию softmax_loss_naive и реализуйте (наивную)
# версию градиента с помощью циклов.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# Как и для SVM, используйте числовой градиент для проверки.
# Их значения должны быть близки.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# Делаем ещё одну проверку с регуляризацией (как и в SVM)
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

In [ ]:
# Теперь реализуйте векторизованную версию функции потерь softmax_loss_vectorized.
# Обе версии должны давать одинаковый результат, но векторизация должна работать быстрее.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('Потери с помощью циклов: %e время на вычисление %fс' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('Векторизованные потери: %e время на вычисление %fс' % (loss_vectorized, toc - tic))

# Как и для SVM, используем норму Фробениуса для сравнения градиентов.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Разница между потерями: %f' % np.abs(loss_naive - loss_vectorized))
print('Разница между градиентами: %f' % grad_difference)

In [ ]:
# Используйте оценочную выборку, чтобы настроить гиперпараметры (силу регялризации
# и скорость обучения). Поэкспереминтируйте с различными значениями; вы должны получить
# точность классификации около 0.35 на оценочной выборке.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Используйте оценочную выборку, чтобы настроить скорость обучения и силу      #
# регуляризации. Этот процесс будет идентичным тому, что вы проделывали в      #
# задании с SVM. Сохраните лучший softmax классификатор в переменную           #
# best_softmax.                                                                #
################################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****
    
# Выводим результаты.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('Скорость обучения: %e регуляризация: %e точность обучения: %f точность оценки: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('Лучшая точность, достигнутая в результате кросс-валидации: %f' % best_val)

In [ ]:
# Оцениваем классификатор на тестовой выборке
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('Окончательная точность классификатора softmax на тестовой выборке: %f' % (test_accuracy, ))

In [ ]:
# Визуализируем веса для каждого класса
w = best_softmax.W[:-1,:] # убираем смещение
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Масштабируем веса в диапазон 0...255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])